In [ ]:
import pandas as pd
import numpy as np
import os

### Main trip-related and demographic information

In [2]:
fname = 'survey_standard_2021-03-22.csv'

df_raw = pd.read_csv(os.path.join('M:\\Data\\OnBoard\\Data and Reports\\_data Standardized', fname), encoding = 'ISO-8859-1')

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,7,9,22,23,26,27,28,29,30,32,40,41,42,52,55,56) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df_raw.copy()
df['operator_survey_year'] = df['operator'] + ' - ' + df['survey_year'].apply(lambda x: str(x))

print('Read {} rows from the data including the following surveys: \n{}'.format(df.shape[0],
                                                                              list(df.operator_survey_year.unique())))

display(df.head())

Read 146457 rows from the data including the following surveys: 
['AC Transit - 2018', 'ACE - 2019', 'BART - 2015', 'Caltrain - 2014', 'Capitol Corridor - 2019', 'City Coach - 2017', 'County Connection - 2019', 'Delta Breeze - 2017', 'FAST - 2017', 'Golden Gate Transit - 2018', 'LAVTA - 2018', 'Marin Transit - 2017', 'Napa Vine - 2014', 'Napa Vine - 2019', 'Petaluma Transit - 2018', 'Santa Rosa CityBus - 2018', 'SF Muni - 2017', 'Soltrans - 2017', 'Sonoma-Marin Area Rail Transit - 2018', 'Sonoma County Transit - 2018', 'TriDelta - 2019', 'Union City Transit - 2017', 'VTA - 2017', 'WestCAT - 2017', 'WETA - 2019']


,ID,operator,survey_year,survey_tech,access_mode,alt_weight,depart_hour,depart_time,dest_purp,direction,...,home_maz,orig_maz,school_maz,workplace_maz,board_tap,alight_tap,trip_weight,field_language,survey_time,operator_survey_year
0,1,AC Transit,2018,local bus,walk,NaN,NaN,NaN,work,NaN,...,11514.0,11514.0,NaN,425106.0,90131.0,490370.0,1.855557,ENGLISH,17:30:00,AC Transit - 2018
1,10,AC Transit,2018,local bus,walk,NaN,NaN,NaN,work,NaN,...,316876.0,316876.0,NaN,324992.0,390602.0,390316.0,4.847912,ENGLISH,07:30:00,AC Transit - 2018
2,100,AC Transit,2018,local bus,walk,NaN,NaN,NaN,home,NaN,...,318470.0,318567.0,NaN,318567.0,390526.0,390132.0,9.772444,ENGLISH,17:30:00,AC Transit - 2018
3,1000,AC Transit,2018,local bus,walk,NaN,NaN,NaN,work,NaN,...,327175.0,327175.0,NaN,325899.0,390316.0,390746.0,10.572424,ENGLISH,08:30:00,AC Transit - 2018
4,10000,AC Transit,2018,local bus,walk,NaN,7.0,NaN,work-related,NaN,...,324492.0,323825.0,NaN,311933.0,390086.0,390449.0,1.270000,ENGLISH,11:30:00,AC Transit - 2018


In [4]:
# consistently label 'missing' value for fields that should have a value
missing_value_dict = {'missing': 'missing',
                      'MISSING': 'missing',
                      'Missing': 'missing'}

for colname in ['race', 'hispanic', 'household_income', 'approximate_age', 'work_status', 'student_status', 'auto_suff',
                'access_mode', 'egress_mode', 'boardings',  'depart_hour', 'return_hour',
                'tour_purp', 'weekpart', 'day_part', 
                'fare_medium', 'fare_category', 'eng_proficient',
                'persons', 'gender', 'worker_numeric_cat', 'vehicle_numeric_cat']:
    df[colname+'_temp'] = df[colname].map(missing_value_dict)
    df.loc[df[colname+'_temp'] == 'missing', colname] = 'missing'
    df.loc[df[colname].isnull(), colname] = 'missing'

    missing = df.loc[df[colname] == 'missing']
    print('{} has missing value in {} rows, {:.1%} of total'.format(colname, missing.shape[0], missing.shape[0]/df.shape[0]))
    
    df.drop(columns = [colname+'_temp'], inplace=True)

race has missing value in 0 rows, 0.0% of total
hispanic has missing value in 4615 rows, 3.2% of total
household_income has missing value in 30497 rows, 20.8% of total
approximate_age has missing value in 7659 rows, 5.2% of total
work_status has missing value in 16857 rows, 11.5% of total
student_status has missing value in 9728 rows, 6.6% of total
auto_suff has missing value in 7351 rows, 5.0% of total
access_mode has missing value in 6120 rows, 4.2% of total
egress_mode has missing value in 6128 rows, 4.2% of total
boardings has missing value in 0 rows, 0.0% of total
depart_hour has missing value in 86247 rows, 58.9% of total
return_hour has missing value in 91904 rows, 62.8% of total
tour_purp has missing value in 3390 rows, 2.3% of total
weekpart has missing value in 27 rows, 0.0% of total
day_part has missing value in 5907 rows, 4.0% of total
fare_medium has missing value in 6348 rows, 4.3% of total
fare_category has missing value in 5944 rows, 4.1% of total
eng_proficient has mis

In [5]:
# summarize access and egress modes
df['access_egress_modes'] = df['access_mode'] + '-' + df['egress_mode']
# display(df.access_egress_modes.value_counts())

access_egress_mode_dict = {'walk-walk': 'walk at both ends',
                           'pnr-walk': 'pnr at one end',
                           'knr-walk': 'knr at one end',
                           'walk-pnr': 'pnr at one end',
                           'walk-knr': 'knr at one end',
                           'bike-bike': 'bike at both ends',
                           'missing-missing': 'missing at least one end',
                           'pnr-pnr': 'pnr at both ends',
                           'knr-knr': 'knr at both ends',
                           'knr-pnr': 'pnr and knr',
                           'pnr-knr': 'pnr and knr',
                           'bike-walk': 'bike at one end',
                           'walk-bike': 'bike at one end',
                           'tnc-walk': 'tnc at one end',
                           'walk-tnc': 'tnc at one end',
                           'bike-pnr': 'pnr at one end',
                           'pnr-bike': 'pnr at one end',
                           'bike-knr': 'knr at one end',
                           'knr-bike': 'knr at one end',
                           'walk-missing': 'missing at least one end',
                           'missing-walk': 'missing at least one end',
                           'other-walk': 'other',
                           'tnc-tnc': 'tnc at both ends',
                           'pnr-tnc': 'pnr and tnc',
                           'knr-tnc': 'pnr and tnc',
                           'tnc-knr': 'pnr and tnc',
                           'tnc-pnr': 'pnr and tnc',
                           'walk-other': 'other',
                           'missing-pnr': 'missing at least one end',
                           'pnr-missing': 'missing at least one end',
                           'missing-knr': 'missing at least one end',
                           'bike-tnc': 'tnc at one end',
                           'knr-missing': 'missing at least one end',
                           'bike-missing': 'missing at least one end',
                           'tnc-bike': 'tnc at one end',
                           'knr-other': 'knr at one end',
                           'other-other': 'other',
                           'other-pnr': 'pnr at one end',
                           'other-knr': 'knr at one end',
                           'other-bike': 'bike at one end',
                           'missing-tnc': 'missing at least one end',
                           'missing-other':'missing at least one end',
                           'tnc-missing': 'missing at least one end'}

df['access_egress_modes'] = df['access_egress_modes'].map(access_egress_mode_dict)
display(df.access_egress_modes.value_counts())

walk at both ends           95236
pnr at one end              18903
knr at one end              13148
missing at least one end     6242
bike at both ends            4452
pnr and knr                  2583
pnr at both ends             1932
knr at both ends             1567
bike at one end              1505
tnc at one end                595
pnr and tnc                   161
other                          85
tnc at both ends               48
Name: access_egress_modes, dtype: int64

In [6]:
# create 'age_group' field

df.loc[df.approximate_age == 'missing', 'approximate_age'] = np.nan
df['approximate_age'] = df['approximate_age'].fillna(0)
df['approximate_age'] = df['approximate_age'].apply(lambda x: int(x))
df['age_group'] = 'missing'
df.loc[(df.approximate_age > 0) & (df.approximate_age < 16), 'age_group'] = 'below than 16'
df.loc[(df.approximate_age > 15) & (df.approximate_age < 23), 'age_group'] = '16 to 22'
df.loc[(df.approximate_age > 22) & (df.approximate_age < 30), 'age_group'] = '23 to 29'
df.loc[(df.approximate_age > 29) & (df.approximate_age < 40), 'age_group'] = '30 to 39'
df.loc[(df.approximate_age > 39) & (df.approximate_age < 50), 'age_group'] = '40 to 49'
df.loc[(df.approximate_age > 49) & (df.approximate_age < 60), 'age_group'] = '50 to 59'
df.loc[(df.approximate_age > 59) & (df.approximate_age < 70), 'age_group'] = '60 to 69'
df.loc[df.approximate_age > 69, 'age_group'] = 'Above 69'
display(df.age_group.value_counts())

30 to 39         35700
23 to 29         31616
40 to 49         22629
16 to 22         18328
50 to 59         16404
60 to 69          9653
missing           7659
Above 69          3703
below than 16      765
Name: age_group, dtype: int64

In [7]:
# summarize fare medium and fare category

# use standard value dictionary
dict_df = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\onboard-surveys\util\standard_variable_dict.csv',
                      usecols = ['generic_variable',
                                 'valid_values_for_categoric_variables',
                                 'standard_values_for_categoric_variables'])
dict_df.columns = ['variable_name','value_details','value_summary']

for colname in ['fare_medium', 'fare_category']:
    df[colname] = df[colname].apply(lambda x: x.strip())
    value_dict = dict_df.loc[dict_df.variable_name == colname]
    df = df.merge(value_dict, left_on=colname, right_on='value_details', how='left')
    df.rename(columns = {'value_summary': colname+'_summary'}, inplace=True)
    display(df[colname+'_summary'].value_counts())

clipper (cash)    55944
clipper (pass)    33499
cash              21560
ticket            19958
pass               6977
missing            6348
e-ticket           1760
other               399
transfer             12
Name: fare_medium_summary, dtype: int64

adult              124774
missing              5944
senior               5883
youth                4085
disabled             2582
monthly               774
one-way               664
round-trip            651
other discount        497
other                 415
senior/disabled       188
Name: fare_category_summary, dtype: int64

In [8]:
# export needed fields
basic_info = ['operator_survey_year', 'weekpart', 'day_part', 'trip_weight', 'weight']

trip_info = ['access_egress_modes', 'access_mode', 'egress_mode', 'tour_purp', 'boardings',
             'fare_category_summary', 'fare_medium_summary',
             'commuter_rail_present', 'heavy_rail_present', 'ferry_present', 'light_rail_present', 'express_bus_present']

demo_info = ['persons', 'work_status', 'student_status', 'age_group', 'gender', 'race', 'hispanic', 'eng_proficient', 
             'household_income', 'worker_numeric_cat', 'vehicle_numeric_cat', 'auto_suff']

spatial_info = ['orig_taz', 'dest_taz', 'home_taz', 'workplace_taz', 'school_taz',
#                 'orig_maz', 'dest_maz', 'home_maz', 'workplace_maz', 'school_maz',
                'first_board_lat', 'first_board_lon', 'last_alight_lat', 'last_alight_lon']

In [9]:
df_summary = df[basic_info + trip_info + demo_info + spatial_info]

# check NA in columns
for i in list(df_summary):
    if df_summary[i].isnull().values.any():
        print('column with NA: {}'.format(i))
    if 'missing' in (df_summary[i].unique()):
        print('column with missing: {}'.format(i))

for i in ['work_status', 'student_status']:
    df_summary.loc[df[i] == 'missing', i] = np.nan
    


column with missing: weekpart
column with missing: day_part
column with missing: access_mode
column with missing: egress_mode
column with missing: tour_purp


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys


column with missing: fare_category_summary
column with missing: fare_medium_summary
column with missing: persons
column with missing: work_status
column with missing: student_status
column with missing: age_group
column with missing: gender
column with missing: hispanic
column with missing: eng_proficient
column with missing: household_income
column with missing: worker_numeric_cat
column with missing: vehicle_numeric_cat
column with missing: auto_suff
column with NA: orig_taz
column with NA: dest_taz
column with NA: home_taz
column with NA: workplace_taz
column with NA: school_taz
column with NA: first_board_lat
column with NA: first_board_lon
column with NA: last_alight_lat
column with NA: last_alight_lon


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [10]:
# convert taz data to integer
for i in ['orig_taz', 'dest_taz', 'home_taz', 'workplace_taz', 'school_taz']:
    df_summary[i] = df_summary[i].fillna(0)
    df_summary[i] = df_summary[i].apply(lambda x: int(x))

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [11]:
# export
new_fname = fname.split('.')[0] + '_forTableau.csv'
df_summary.to_csv(os.path.join('M:\\Data\\OnBoard\\Data and Reports\\_data Standardized', new_fname), index=False)

In [12]:
df_summary.loc[df_summary.persons=='missing'].operator_survey_year.value_counts()

BART - 2015                              4053
Caltrain - 2014                           328
Capitol Corridor - 2019                   189
SF Muni - 2017                             27
WestCAT - 2017                             23
Golden Gate Transit - 2018                 15
FAST - 2017                                 8
Marin Transit - 2017                        7
Soltrans - 2017                             6
ACE - 2019                                  5
Napa Vine - 2019                            3
Sonoma-Marin Area Rail Transit - 2018       3
City Coach - 2017                           3
Sonoma County Transit - 2018                2
Union City Transit - 2017                   1
Name: operator_survey_year, dtype: int64

In [13]:
df_summary.loc[df_summary.gender=='missing'].operator_survey_year.value_counts()

BART - 2015                   4009
Caltrain - 2014                328
SF Muni - 2017                 121
AC Transit - 2018              117
Capitol Corridor - 2019        115
Golden Gate Transit - 2018      36
VTA - 2017                      27
Marin Transit - 2017            20
Soltrans - 2017                  4
Petaluma Transit - 2018          1
Name: operator_survey_year, dtype: int64

In [14]:
df_summary.loc[df_summary.worker_numeric_cat=='missing'].operator_survey_year.value_counts()

BART - 2015                              4391
Capitol Corridor - 2019                  2540
Caltrain - 2014                           328
SF Muni - 2017                             27
WestCAT - 2017                             15
Marin Transit - 2017                        8
ACE - 2019                                  3
Napa Vine - 2019                            3
Sonoma-Marin Area Rail Transit - 2018       2
Sonoma County Transit - 2018                2
Union City Transit - 2017                   1
Name: operator_survey_year, dtype: int64

In [15]:
df_summary.loc[df_summary.vehicle_numeric_cat=='missing'].operator_survey_year.value_counts()

BART - 2015                              4009
Capitol Corridor - 2019                  2540
Caltrain - 2014                           328
SF Muni - 2017                             27
Golden Gate Transit - 2018                 14
WestCAT - 2017                             11
ACE - 2019                                  3
Marin Transit - 2017                        1
Union City Transit - 2017                   1
Napa Vine - 2019                            1
Sonoma-Marin Area Rail Transit - 2018       1
Sonoma County Transit - 2018                1
WETA - 2019                                 1
Name: operator_survey_year, dtype: int64

In [16]:
df_summary.loc[df_summary.work_status.isnull()].operator_survey_year.value_counts()

VTA - 2017                      8754
BART - 2015                     3297
Capitol Corridor - 2019         2540
SF Muni - 2017                  1479
Caltrain - 2014                  328
Golden Gate Transit - 2018       258
TriDelta - 2019                   84
LAVTA - 2018                      55
Petaluma Transit - 2018           30
Marin Transit - 2017              28
Sonoma County Transit - 2018       3
WestCAT - 2017                     1
Name: operator_survey_year, dtype: int64

In [17]:
df_summary.loc[df_summary.student_status.isnull()].operator_survey_year.value_counts()

BART - 2015                     4453
Capitol Corridor - 2019         2540
SF Muni - 2017                  1702
VTA - 2017                       680
Caltrain - 2014                  328
Marin Transit - 2017              18
Union City Transit - 2017          2
Napa Vine - 2014                   2
Sonoma County Transit - 2018       2
Golden Gate Transit - 2018         1
Name: operator_survey_year, dtype: int64

In [18]:
df_summary.operator_survey_year.value_counts()

SF Muni - 2017                           48334
BART - 2015                              46948
VTA - 2017                               15394
AC Transit - 2018                        14678
Caltrain - 2014                           6032
Golden Gate Transit - 2018                4103
Capitol Corridor - 2019                   2540
Marin Transit - 2017                      1216
County Connection - 2019                  1188
TriDelta - 2019                            742
Santa Rosa CityBus - 2018                  674
LAVTA - 2018                               554
WETA - 2019                                550
Napa Vine - 2014                           452
Sonoma-Marin Area Rail Transit - 2018      411
Soltrans - 2017                            398
Sonoma County Transit - 2018               395
WestCAT - 2017                             384
Napa Vine - 2019                           338
ACE - 2019                                 333
Petaluma Transit - 2018                    286
FAST - 2017  

### Origin TAZ, Destination TAZ, Home TAZ, Workplace TAZ, School TAZ 

In [19]:
orig_taz = df_summary.groupby(['operator_survey_year', 'weekpart', 'day_part', 'orig_taz'])['trip_weight'].sum().reset_index()
orig_taz.rename(columns = {'trip_weight': 'trip_weight_by_orig_taz',
                           'orig_taz': 'TM2_taz'}, inplace=True)
print(orig_taz.shape)
print(list(orig_taz))

dest_taz = df_summary.groupby(['operator_survey_year', 'weekpart', 'day_part', 'dest_taz'])['trip_weight'].sum().reset_index()
dest_taz.rename(columns = {'trip_weight': 'trip_weight_by_dest_taz',
                           'dest_taz': 'TM2_taz'}, inplace=True)
print(dest_taz.shape)
print(list(dest_taz))

home_taz = df_summary.groupby(['operator_survey_year', 'weekpart', 'day_part', 'home_taz'])['trip_weight'].sum().reset_index()
home_taz.rename(columns = {'trip_weight': 'trip_weight_by_home_taz',
                           'home_taz': 'TM2_taz'}, inplace=True)
print(home_taz.shape)
print(list(home_taz))

workplace_taz = df_summary.groupby(['operator_survey_year', 'weekpart', 'day_part', 'workplace_taz'])['trip_weight'].sum().reset_index()
workplace_taz.rename(columns = {'trip_weight': 'trip_weight_by_workplace_taz',
                                'workplace_taz': 'TM2_taz'}, inplace=True)
print(workplace_taz.shape)
print(list(workplace_taz))

school_taz = df_summary.groupby(['operator_survey_year', 'weekpart', 'day_part', 'school_taz'])['trip_weight'].sum().reset_index()
school_taz.rename(columns = {'trip_weight': 'trip_weight_by_school_taz',
                             'school_taz': 'TM2_taz'}, inplace=True)
print(school_taz.shape)
print(list(school_taz))


(30172, 5)
['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz', 'trip_weight_by_orig_taz']
(28927, 5)
['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz', 'trip_weight_by_dest_taz']
(33206, 5)
['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz', 'trip_weight_by_home_taz']
(19999, 5)
['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz', 'trip_weight_by_workplace_taz']
(3735, 5)
['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz', 'trip_weight_by_school_taz']


In [20]:
TM2_taz_summary = orig_taz.merge(dest_taz,
                                 on=['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz'],
                                 how='outer').merge(home_taz,
                                                    on=['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz'],
                                                    how='outer').merge(workplace_taz,
                                                                       on=['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz'],
                                                                       how='outer').merge(school_taz,
                                                                                          on=['operator_survey_year', 'weekpart', 'day_part', 'TM2_taz'],
                                                                                          how='outer')
for i in ['trip_weight_by_orig_taz', 'trip_weight_by_dest_taz',
          'trip_weight_by_home_taz', 'trip_weight_by_workplace_taz', 'trip_weight_by_school_taz']:
    TM2_taz_summary[i] = TM2_taz_summary[i].fillna(0)

In [21]:
taz_fname = fname.split('.')[0] + '_TM2_taz_summary.csv'
TM2_taz_summary.to_csv((os.path.join('M:\\Data\\OnBoard\\Data and Reports\\_data Standardized', taz_fname)), index=False)

In [22]:
TM2_taz_summary.loc[TM2_taz_summary.TM2_taz > 0].groupby(['operator_survey_year', 'weekpart', 'TM2_taz']).trip_weight_by_home_taz.sum().reset_index().sort_values(by='trip_weight_by_home_taz', ascending=False)

,operator_survey_year,weekpart,TM2_taz,trip_weight_by_home_taz
11923,SF Muni - 2017,WEEKDAY,71,3554.862125
12394,SF Muni - 2017,WEEKDAY,553,3493.818897
12397,SF Muni - 2017,WEEKDAY,556,3445.389347
12087,SF Muni - 2017,WEEKDAY,237,3395.492763
12425,SF Muni - 2017,WEEKDAY,586,3210.173972
...,...,...,...,...
10398,Golden Gate Transit - 2018,WEEKEND,700282,0.000000
10399,Golden Gate Transit - 2018,WEEKEND,700283,0.000000
10400,Golden Gate Transit - 2018,WEEKEND,700284,0.000000
10401,Golden Gate Transit - 2018,WEEKEND,700285,0.000000


In [23]:
TM2_taz_summary.loc[TM2_taz_summary.TM2_taz > 0].groupby(['operator_survey_year','weekpart', 'TM2_taz']).trip_weight_by_workplace_taz.sum().reset_index().sort_values(by='trip_weight_by_workplace_taz', ascending=False)

,operator_survey_year,weekpart,TM2_taz,trip_weight_by_workplace_taz
12432,SF Muni - 2017,WEEKDAY,597,19534.432951
12426,SF Muni - 2017,WEEKDAY,588,18222.533085
3327,BART - 2015,WEEKDAY,588,18157.782493
12431,SF Muni - 2017,WEEKDAY,595,17133.279363
3157,BART - 2015,WEEKDAY,404,16658.923241
...,...,...,...,...
11042,Marin Transit - 2017,WEEKDAY,800111,0.000000
11043,Marin Transit - 2017,WEEKDAY,800113,0.000000
11044,Marin Transit - 2017,WEEKDAY,800115,0.000000
11045,Marin Transit - 2017,WEEKDAY,800116,0.000000


In [24]:
TM2_taz_summary.loc[TM2_taz_summary.TM2_taz > 0].groupby(['operator_survey_year','weekpart', 'TM2_taz']).trip_weight_by_school_taz.sum().reset_index().sort_values(by='trip_weight_by_school_taz', ascending=False)

,operator_survey_year,weekpart,TM2_taz,trip_weight_by_school_taz
12475,SF Muni - 2017,WEEKDAY,640,11610.023648
17806,VTA - 2017,WEEKDAY,200920,10711.554800
1410,AC Transit - 2018,WEEKDAY,301060,7343.068707
2907,BART - 2015,WEEKDAY,133,6799.663125
5163,BART - 2015,WEEKDAY,301060,6714.003049
...,...,...,...,...
7075,Caltrain - 2014,WEEKDAY,200334,0.000000
7074,Caltrain - 2014,WEEKDAY,200331,0.000000
7073,Caltrain - 2014,WEEKDAY,200326,0.000000
7072,Caltrain - 2014,WEEKDAY,200325,0.000000
